In [8]:
import sys
import importlib
import os
# make the module available from the src directory
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [9]:
from src.dataLoaders.PatientDataLoader import PatientDataProcessor
importlib.reload(sys.modules['src.dataLoaders.PatientDataLoader'])

<module 'src.dataLoaders.PatientDataLoader' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/PatientDataLoader.py'>

In [10]:
meta_folder = 'OAIData21/'
data_path = '../datasets/meta_data/' + meta_folder

processor = PatientDataProcessor(base_path=data_path)
meta_data = processor.load_all_clinical_data(labels=["WOMKP"])

In [4]:
meta_data

,V00AGE,V00WOMKPL,V00WOMKPR,Sex,V01AGE,V01WOMKPL,V01WOMKPR,Sex,V02AGE,V02WOMKPL,...,V09WOMKPR,Sex,V10AGE,V10WOMKPL,V10WOMKPR,Sex,V11AGE,V11WOMKPL,V11WOMKPR,Sex
ID,,,,,,,,,,,,,,,,,,,,,
9000099,59,0.0,1.0,0,60.0,0.0,0.0,0,NaN,NaN,...,0.0,0,66.0,4.0,3.0,0,67.0,4.0,2.0,0
9000296,69,0.0,0.0,0,70.0,0.0,0.0,0,NaN,NaN,...,0.0,0,77.0,0.0,0.0,0,78.0,0.0,0.0,0
9000622,71,0.0,3.0,1,72.0,0.0,2.0,1,NaN,NaN,...,,1,NaN,NaN,NaN,1,NaN,NaN,NaN,1
9000798,56,8.0,0.0,0,58.0,8.0,0.0,0,58.0,7.0,...,0.0,0,64.0,5.0,0.0,0,65.0,3.0,0.0,0
9001104,72,0.0,4.0,1,73.0,0.0,1.0,1,NaN,NaN,...,,1,80.0,0.0,1.0,1,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999365,56,4.0,5.0,0,57.0,3.0,5.0,0,NaN,NaN,...,1.0,0,64.0,4.0,2.0,0,65.0,5.0,6.0,0
9999510,50,3.0,0.0,0,51.0,1.0,0.0,0,NaN,NaN,...,1.0,0,58.0,3.0,0.0,0,59.0,4.0,0.0,0
9999862,61,0.0,0.0,1,62.0,0.0,0.0,1,NaN,NaN,...,0.0,1,69.0,0.0,0.0,1,70.0,0.0,0.0,1


In [11]:
from src.dataLoaders.utils.BalancedDataLoader import BalancedBatchSampler
importlib.reload(sys.modules['src.dataLoaders.utils.BalancedDataLoader'])

from src.dataLoaders.NiftiDataLoader2 import NiftiDataLoader
importlib.reload(sys.modules['src.dataLoaders.NiftiDataLoader2'])

from src.transformers.SliceTransformer import SliceAggregateTransform
importlib.reload(sys.modules['src.transformers.SliceTransformer'])
custom_transforms = [SliceAggregateTransform(keys=["image"], slices=9)]

In [17]:
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstD, ScaleIntensityd, ToTensord, ResizeD, Lambdad, NormalizeIntensityD

importlib.reload(sys.modules['src.transformers.TensorLoaderTransfomer'])
transforms = [
            LoadImaged(keys=["image"], ensure_channel_first=True),
            # Lambdad(keys=["image"], func=lambda x: x.half()),  # Convert to float16
            # ResizeD(keys=["image"], spatial_size=3),
            ScaleIntensityd(keys=["image"]),
            # NormalizeIntensityD(keys=["image"], nonzero=True, channel_wise=True),
        ]


In [18]:
dataset = 'niftiShort'
data_path = '../datasets/nifti/'
data_dir = f'{data_path}{dataset}'



# data_dir = 'C:/Users/askel/Downloads/NIFTY/NIFTY/'
# max = 8876
dim = 384
data_loader = NiftiDataLoader(data_dir=data_dir, 
                              meta_data_loader=processor,
                              batch_size=64,
                              spatial_resize=(dim, dim, dim),
                              cache_rate=0.5, 
                              replace_rate=1,
                              transforms=transforms,
                              )
# data_loader.load_data(visit_no=visit, subset_size=2058, cache="standard")
# data_loader.load_data(subset_size=84, cache="standard")
#data_loader.load_data(subset_size=4400, cache="standard")
# data_loader.load_data(subset_size=8875, cache="standard")
# data_loader.load_data(cache="persistent")
data_loader.load_data(cache="standard")

File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Data list loaded: False
File /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/dataLoaders/saved_data_lists/data_list.pkl does not exist.
Visits: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
Total images detected: 83
Subset size: 83


Loading dataset: 100%|██████████| 8/8 [00:01<00:00,  4.14it/s]

batch_size: 64
Target distribution: {0.0: 8, 1.0: 8, 2.0: 8, 3.0: 8, 4.0: 8, 5.0: 8, 7.0: 8, 8.0: 8}


In [39]:
from src.models.medical_models.monai_resnet import MonaiMedicalResNet
importlib.reload(sys.modules['src.models.medical_models.monai_resnet'])
importlib.reload(sys.modules['src.models.medical_models.base_medical'])

<module 'src.models.medical_models.monai_resnet' from '/Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/monai_resnet.py'>

In [8]:
len(data_loader.train_loader.dataset[0]['label'])

1

In [41]:
# Create a model instance
model = MonaiMedicalResNet(
    num_epochs=50,
    learning_rate=0.001,
    weight_decay=1e-4,
    data_loader=data_loader,
    dropout_rate=0.1,
    depth=18,
    pretrained=True
)

# model.load_model('MedicalResNetModel_18_6212_epoch_38.pth')
model.train()
model.evaluate()

Data loader train loader:  1 50
Class counts: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 7.0: 0}
Image spatial dimensions:  torch.Size([20, 9, 384, 384])
Number of input channels:  9
gpu:  mps:0
Is cuda available:  False mps
Number of training images: 50
Class counts: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 7.0: 0}
Epoch 1/50, Train Loss: 6.432741641998291, Val Loss: 16.8870906829834, R^2 Score: -1.2944416999816895
Model saved at /Users/askelundsgaard/Documents/datalogi/6-semester/Bachelor/XAI-in-Medical-Image-Analysis/src/models/medical_models/saved_models/MedicalResNetModel_18_50_height_384_epoch_1_val_16.89_r2_16.89.pth
Class counts: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 7.0: 0}
Epoch 2/50, Train Loss: 3.459548234939575, Val Loss: 20.88651466369629, R^2 Score: -1.8378419876098633
Class counts: {0.0: 0, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0, 5.0: 0, 7.0: 0}
Epoch 3/50, Train Loss: 49.777801513671875, Val Loss: 9.834564208984375, R^2 Score: -0.336217999458313
Model 

In [ ]:
# evaluate with the best model
# model.load_model('MedicalResNetModel_18_487_height_384_epoch_34_val_0.13.pth')
# model.evaluate()